In [9]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacUser"
password = "wonky"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readAll({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(className='row',
         style={'display' : 'flex'},
         children=[
             html.A([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                     style={
                         'height' : '100px',
                         'width' : '150px',
                         'float' : 'left',
                         'position' : 'relative',
                         'padding-top' : 0,
                         'padding-right' : 0
                     })],
              href='https://www,snhu.edu'),
              html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
              html.Img(id="BScott", src="https://townsquare.media/site/690/files/2022/02/attachment-Blazing-Saddles-Alex-Karras.jpg?w=1200&h=0&zc=1&s=0&a=t&q=89", style={'height':'100px', 'width':'100px', 'float':'right', 'position':'relative'}),
         ]),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.Dropdown(
            id='dogFilterDropdown',
            options = [{'label': 'All', 'value': 'All'},
                       {'label': 'Water Rescue', 'value': 'waterRescue'},
                       {'label': 'Mountain or Wilderness Rescue', 'value':'mountainRescue'},
                       {'label': 'Disaster Rescue', 'value':'disasterRescue'}],
        value='All')
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_rows = [],
        selected_columns = [],
        page_action="native",
        page_current=0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
         children=[
             html.Div(
                id='graph-id',
                className='col s12 m6',

             ),
             html.Div(
                id='map-id',
                className='col s12 m6',
             )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('dogFilterDropdown', 'value')])
def update_dashboard(value):
### FIX ME Add code to filter interactive data table with MongoDB queries
    if(value == 'All'):
        df=pd.DataFrame.from_records(shelter.readAll({}))
    elif(value == 'waterRescue'):
        df=pd.DataFrame(list(shelter.readAll({'breed' : {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]}, "sex_upon_outcome" : "Intact Female", "age_upon_outcome_in_weeks":{"$lte":156, "$gte":26}})))
    elif(value == "mountainRescue"):
        df=pd.DataFrame(list(shelter.readAll({'breed' : {"$in": ["German Shepherd","Alaskan Malamute","Old English Sheepdog", "Siberian Husky", "Rottwiler"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks":{"$lte":156, "$gte":26}})))
    elif(value == "disasterRescue"):
        df=pd.DataFrame(list(shelter.readAll({'breed' : {"$in": ["Doberman Pinscher","German Sheperd","Golden Retriever", "Bloodhound", "Rottweiler"]}, "sex_upon_outcome" : "Intact Male", "age_upon_outcome_in_weeks":{"$lte":300, "$gte":20}})))
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return(data, columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data')])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            id='graph_id',
            figure = px.pie(dff,
                         values='age_upon_outcome_in_weeks',
                         names='breed',
                         hole=0.5)
        )   
    ]

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data')])
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    return[
        dl.Map(style={'width':'1000px', 'height':'500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ]


app